## Parsing the data

In the "Data_Download_Extraction.ipynb' We downloaded the multiple causes of death data files (2012-2016) form the CDC and unzipped them into the "data" directory under the current working directory.

Here we will try to parse the data from the extracted files into one giant CSV based on this document: https://www.cdc.gov/nchs/data/dvs/Multiple_Cause_Record_Layout_2016.pdf

In [1]:
import os

cwd = os.getcwd()
years = list(range(2012, 2017))
data_file_names = []
for _1, _2, files  in os.walk(cwd+"\\data"):
    data_file_names = files
data_file_names = {year:file for year, file in zip(years, data_file_names)}

In [28]:
#desired fields in the data

fields = ["year", "month", "age", "sex", "race", "cause", "place", "education"]

In [27]:
Manner = {1: "Accident",
2: "Suicide",
3: "Homicide",
4: "Pending investigation",
5: "Could not determine",
6: "Self-Inflicted",
7: "Natural",} 

ICD_10_Suicide_Range = ["X"+str(num) for num in range(60,85)]

suicide_cause_dict = {}

for item in ICD_10_Suicide_Range:
    num = int(item[1:])
    cause = ""
    if num in range(60, 70):
        cause =  "poisoning"
    elif num is 70:
        cause = "hanging/suffocation"
    elif num is 71:
        cause = "drowning"
    elif num in range(72,75):
        cause = "gun"
    elif num is 75:
        cause = "explosive"
    elif num in [76, 77]:
        cause = "burning/burns"
    elif num is 78:
        cause = "sharp object"
    elif num is 79:
        cause = "blunt object"
    elif num is 80:
        cause = "jumping"
    elif num is 81:
        cause = "moving object"
    elif num is 82:
        cause = "crashing vehicle"
    else:
        cause = "other"
    suicide_cause_dict[item] = cause

Let us first extract every death marked as a suicied into a list of strings
ICD 10 codes for suicides are found here (X60 - X84):https://en.wikipedia.org/wiki/ICD-10_Chapter_XX:_External_causes_of_morbidity_and_mortality

the manner field may be blank, and the cause of death may not be reported as an ICD code. But hopefully using both, we will arrive at a resonable tally of all sucides in the US in the five year period of 2012 to 2016

In [24]:
suicide_deaths = []

for file in data_file_names.values():
    
    with open(cwd+"\\data\\"+file) as f:
        for line in f:
            
            manner = line[106].strip()
            ICD10 = line[145:149].strip()
            
            if manner and int(manner) in [2, 6]:
                suicide_deaths.append(line)
                
            elif ICD10 and ICD10 in ICD_10_Suicide_Range:
                suicide_deaths.append(line)
                

In [26]:
len(suicide_deaths)

215211

**215211** sucicides in five years! That is over a 117 suicids per day.

We will now process each line in the list to add it to a pandas data frame with the fields specified earlier

In [29]:
import pandas as pd

suicides_df = pd.DataFrame(columns = fields)

In [ ]:
def process_line(line):
    year = line[101:105]
    month = line[]
    